![Rampart Logo](../images/logo.png)

Twinkle is a flat ranking model based on the learning to rank (LTR) technique. [Lambdarank](https://www.microsoft.com/en-us/research/uploads/prod/2016/02/MSR-TR-2010-82.pdf) is used as the main logical engine. The final ranker is supposed to relevantly order housing publications to a final Telegram user. Real-time ranking power can be checked via a local dev service **browsing** .

## I/O
Required data consists of two parts:
- Training tabular data lies in `../data/twinkle/training.csv` ;
- Testing (validation) tabular data lies in `../data/twinkle/testing.csv` .

Output model must be stored into `../models/twinkle.txt` .

## Features
- `actual_price` - true flat's price (in USD);
- `utmost_price` - query's price limit (in USD), search results shouldn't exceed much this shape;
- `total_area` - overall apartment's area (in square meters);
- `living_area` - flat's living room area (in square meters);
- `kitchen_area` - flat's kitchen area (in square meters);
- `actual_room_number` - true flat's living room amount;
- `desired_room_number` - target room count;
- `actual_floor` - apartment's floor (the ground floor is the floor #1);
- `total_floor` - house's floor count;
- `desired_floor` - target flat's floor;
- `housing` - either a newbuild or a used apartments;
- `ssf` - Subway Station Factor, the score indicating about subway stations nearby;
- `izf` - Industrial Zone Factor, the score indicating about factories & plants nearby;
- `gzf` - Green Zone Factor, the score indicating about parks nearby;
- `relevance` - sample's search quality (just for training);
- `query` - sample's group ID (just for training).

## Categorical data
All feature categories are sorted according to their ranks.
- `desired_room_number`
    * `whatever` - not matter how many rooms;
    * `1` - 1 room;
    * `2` - 2 rooms;
    * `3` - 3 rooms;
    * `4+` - huge luxurious apartments with many rooms;
- `desired_floor`
    * `whatever` - not matter what floor;
    * `low` - low floors are preferred;
    * `high` - top floors are preferred;
- `housing`
    * `primary` - newbuilds & houses under construction;
    * `secondary` - old & already used apartments;
- `relevance`
    * `terrible` - don't show this thing again!
    * `bad` - poor quality;
    * `so-so` - average result;
    * `good` - quite smart search;
    * `excellent` - the best matches.

In [ ]:
%matplotlib inline

In [ ]:
from pandas import read_csv
from lightgbm import train, Dataset

In [ ]:
def load(path, params, reference=None):
    frame = read_csv(path)
    return Dataset(
        frame.drop(columns=['relevance', 'query']),
        frame['relevance'],
        group=frame.groupby(['query']).size(),
        categorical_feature=['desired_room_number', 'desired_floor', 'housing'],
        reference=reference,
        params=params
    )

In [ ]:
params = {'objective': 'lambdarank', 'metric': 'ndcg'}
training = load('../data/twinkle/training.csv', params)
testing = load('../data/twinkle/testing.csv', params, training)
train(params, training, 30, [testing], early_stopping_rounds=15).save_model('../models/twinkle.txt')
pass